In [1]:
from astropy.table import Table
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('/pbs/home/c/cmurray/cluster_likelihood/modules/sbi/')
import simulation
import torch
from sbi import utils as utils
from sbi import analysis as analysis
from sbi.inference.base import infer
import corner
import pickle
import scipy.stats as stats
from matplotlib.cm import get_cmap
import pickle
import pyccl as ccl
sys.path.append('/pbs/home/c/cmurray/cluster_likelihood/pinocchio/')
import pinocchio_binning_scheme as binning_scheme
def save_pickle(dat, filename, **kwargs):
    file = open(filename,'wb')
    pickle.dump(dat, file)
    file.close()
def load(filename, **kwargs):
    with open(filename, 'rb') as fin:
        return pickle.load(fin, **kwargs)

ModuleNotFoundError: No module named 'model_halo_abundance'

# Create capish simulators

In [ ]:
stacked_simulator = simulation.Universe_simulation( 'des_stacked_counts' ,
                                                    variable_params=['omega_m', 
                                                                     'sigma_8', 
                                                                     'h',
                                                                     'alpha' , 
                                                                     'B' ,
                                                                     'log10Mmin',
                                                                     'beta',
                                                                     'sigma',
                                                                     'r'],
                                                    fixed_params={'w_0': -1, 'w_a': 0 } )
stacked_simulator.selection_richness = 0
stacked_simulator.dOmega = None
stacked_simulator.richness_bins = np.logspace( np.log10( 10 ) , np.log10( 300 ) , 12 )
stacked_simulator.redshift_bins = np.linspace( 0.2 , 1 , 20 )
stacked_simulator.sigma_mwl = 0.25
stacked_simulator.include_mwl_measurement_errors = False
# measure errors for weak lensing masses
stacked_simulator.mwl_std = 0.3
stacked_simulator.correlation_mass_evolution = False
stacked_simulator.set_richness_mass_relation( 'halo model' )
stacked_simulator.massdef = ccl.halos.massdef.MassDef('200', 'matter',)
stacked_simulator.halobias_fct = ccl.halos.hbias.tinker10.HaloBiasTinker10(mass_def=stacked_simulator.massdef)
# set the Aemulus correction to the halo mass function
stacked_simulator.s = 0.037
stacked_simulator.q = 1.008

stacked_simulator.hmf = ccl.halos.hmfunc.MassFuncTinker08( mass_def ='200m')
stacked_simulator.Z_edges_hybrid = binning_scheme.redshift_edges
Z_bin_hybrid = [[stacked_simulator.Z_edges_hybrid[i], stacked_simulator.Z_edges_hybrid[i+1]] for i in range(len(stacked_simulator.Z_edges_hybrid)-1)]
stacked_simulator.poisson_only = False
stacked_simulator.use_hybrid = False

stacked_simulator.f_sky = 1

# set the binning used to simulate a cluster catalogue
stacked_simulator.set_bins( z_bins = np.arange( 0.15 , 0.7 , 0.05 ) , log10m_bins =  np.arange( 13.1, 16 , 0.01))

In [ ]:
S_ij_full_sky = load(f'/pbs/throng/lsst/users/cpayerne/capish/data/pinocchio_matrix/S_ij_full_sky.pkl' )
stacked_simulator.Sij_SSC = S_ij_full_sky/stacked_simulator.f_sky

sigma_ij_full_sky = load(f'/pbs/throng/lsst/users/cpayerne/capish/data/pinocchio_matrix/sigma_ij_full_sky.pkl' )
stacked_simulator.sigmaij_SSC = sigma_ij_full_sky/stacked_simulator.f_sky

# Counts for different cosmologies

In [ ]:
sim_obs = stacked_simulator.run_simulation( torch.Tensor( [ 0.3,
                                                             0.85,
                                                             0.744,
                                                             0.748 - np.log10(0.744),
                                                             np.log10( 10**12.37 / 10**11.13 ),
                                                             11.13,
                                                             0.0,
                                                             0.31301698088645935, 
                                                             0 ] ))

# Posteriors with and without mass-scaling.